In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score

import tensorflow as tf
import tensorflow.keras
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [78]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype("float32") / 255.0
x_test = x_test.reshape(10000, 784).astype("float32") / 255.0
#x_train, x_test = x_train / 255.0, x_test / 255.0

In [79]:
y_train = y_train.reshape(60000,1)
y_test = y_test.reshape(10000,1)

In [80]:
x_train_tf = tf.data.Dataset.from_tensor_slices(x_train)
x_train_tf = x_train_tf.shuffle(buffer_size=1024).batch(32)


y_train_tf = tf.data.Dataset.from_tensor_slices(y_train)
y_train_tf = y_train_tf.shuffle(buffer_size=1024).batch(32)

In [2]:
batch_size = 32
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = test_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [3]:
class MLP(Model):
    def __init__(self, input_dim, hidden1, hidden2, hidden3, outputs):
        super(MLP, self).__init__()

        self.hidden1 = layers.Dense(hidden1, activation='relu', name='hidden1') # 500
        self.hidden2 = layers.Dense(hidden2, activation='relu', name='hidden2') # 300
        self.hidden3 = layers.Dense(hidden3, activation='relu', name='hidden3') # 100
        self.hidden4 = layers.Dense(10, name='hidden4', activation = 'softmax')
        #self.outputs = layers.Softmax(name = 'output') # 10
        
        #self.droupout = Droupout(0.2)
        self.input_dim = input_dim
    
    def build_graph(self):
        inputs_ = layers.Input(shape=self.input_dim, name = 'inputs')
        return Model(inputs=inputs_, outputs=self.call(inputs_))
        #self._init_graph_network(inputs=self.input_layer,outputs=self.out)
        
#     def model(self):
#         x = layers.Input(shape=self.input_dim)
#         return Model(inputs=x, outputs=self.call(x))

    def call(self, input_data, training=False):
        x = self.hidden1(input_data)
        #if training == True:
            #x = self.droupout(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        x = self.hidden4(x)
        #x = self.outputs(x)
        return x

In [4]:
mlp = MLP(784, 500, 300, 100, 10)

In [21]:
mlp.build_graph().summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
hidden1 (Dense)              (None, 500)               392500    
_________________________________________________________________
hidden2 (Dense)              (None, 300)               150300    
_________________________________________________________________
hidden3 (Dense)              (None, 100)               30100     
_________________________________________________________________
hidden4 (Dense)              (None, 10)                1010      
_________________________________________________________________
output (Softmax)             (None, 10)                0         
Total params: 573,910
Trainable params: 573,910
Non-trainable params: 0
________________________________________________

In [17]:
mlp.summary()

Model: "mlp_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden1 (Dense)              multiple                  392500    
_________________________________________________________________
hidden2 (Dense)              multiple                  150300    
_________________________________________________________________
hidden3 (Dense)              multiple                  30100     
_________________________________________________________________
hidden4 (Dense)              multiple                  1010      
_________________________________________________________________
output (Softmax)             multiple                  0         
Total params: 573,910
Trainable params: 573,910
Non-trainable params: 0
_________________________________________________________________


In [18]:
tf.keras.utils.plot_model(mlp.build_graph(), show_shapes=True, show_layer_names=True, rankdir='TB', dpi=100, expand_nested=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [6]:
np.random.permutation(10)

array([0, 2, 7, 4, 1, 8, 9, 3, 6, 5])

In [5]:
class Trainer:
    def __init__(self,args, train, test):
        self.args = args
        self.train = train
        #self.x_train = train[0]
        #self.y_train = train[1]
        
        self.x_test = test[0]
        self.y_test = test[1]
    
    def train_mlp1(self):
        mlp = MLP(784, 500, 300, 100, 10)
        loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
        metric = tf.keras.metrics.SparseCategoricalAccuracy()
        optimizer = tf.keras.optimizers.Adam()
        #metric = 'sparse_categorical_accuracy'

        mlp.compile(optimizer=optimizer, loss=loss_object, metrics = metric)#metrics=['accuracy'])
        hist = mlp.fit(x=self.x_train, y=self.y_train, batch_size = self.args['batch'], epochs=self.args['epochs'], )
        
        return mlp
    
    
    def train_mlp2(self):
        
        loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
        optimizer = tf.keras.optimizers.Adam()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
        
        mlp = MLP(784, 500, 300, 100, 10)
        
        N = self.x_train.shape[0]
        BS = self.args['batch']
        BN = int(ceil(N / BS))
        
        for epoch in range(self.args['epochs']):
            print(f"Start of epoch {epoch}")
            ind = np.random.permutation(N)
            x_train = self.x_train[ind]
            y_train = self.y_train[ind]
            g_batch = tqdm(range(BN))
            # Iterate over the batches of the dataset.
            loss_tracker = 0
            for step in g_batch:
                x_batch = x_train[step * BS: (step + 1) * BS]
                y_batch = y_train[step * BS: (step + 1) * BS]
                with tf.GradientTape() as tape:
                    output = mlp(x_batch_train)
                    loss = loss_object(y_batch, output)
                    #loss += sum(mlp.losses) 

                grads = tape.gradient(loss, mlp.trainable_weights)
                optimizer.apply_gradients(zip(grads, mlp.trainable_weights))

                train_loss(loss)
                train_accuracy(y_batch_train, output)
                print(train_loss)
                if step % 100 == 0:
                    
                    template = 'Step: {}, 손실: {:.3f}, 정확도: {:.3f}'
                    print(template.format(step,
                                          train_loss.result(),
                                          train_accuracy.result()*100))
                    
                    #print(f"Step: {step}, loss: {train_loss.result()}")
                    #print("step %d: mean loss = %.4f" % (step, train_loss.result()))
            print(f"epoch: {epoch+1}, Loss: {loss_tracker}")
    
    
    def train_mlp(self):
        
        loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
        optimizer = tf.keras.optimizers.Adam()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
        #train_accuracy = tf.keras.metrics.Accuracy(name = 'traom_accuracy')

        
        mlp = MLP(784, 500, 300, 100, 10)
        for epoch in range(self.args['epochs']):
            print(f"Start of epoch {epoch+1}")
            # Iterate over the batches of the dataset.
            loss_tracker = []
            #for step, (x_batch_train, y_batch_train) in enumerate(zip(self.x_train, self.y_train)):
            for step, (x_batch_train, y_batch_train) in enumerate(self.train): 
                with tf.GradientTape() as tape:
                    output = mlp(x_batch_train, training = True)
                    loss = loss_object(y_batch_train, output)
                    #loss += sum(mlp.losses) 

                grads = tape.gradient(loss, mlp.trainable_weights)
                optimizer.apply_gradients(zip(grads, mlp.trainable_weights))
                #train_loss.update_state(loss)
                #train_accuracy.update_state(y_batch_train, output)

                train_loss(loss)
                train_accuracy(y_batch_train, output)
 

                if step % 100 == 0:
                    template = 'Step: {}, 손실: {:.3f}, 정확도: {:.3f}'
                    print(template.format(step,
                                          train_loss.result(),
                                          train_accuracy.result()*100))
                   
            print(f"\
            epoch: {epoch+1},\
            Loss: {train_loss.result():.3f},\
            Accuracy: {train_accuracy.result():.3f},\
            ")
        
                    
    def eval_step(self):
        pass
        #prediction = 

In [6]:
args = {'epochs': 2,
       'batch': 32}

simple_mlp1 = Trainer(args, train=[x_train, y_train], test=[x_test, y_test])
simple_mlp = Trainer(args, train=train_dataset, test=[x_test, y_test])

In [66]:
simple_mlp1.train_mlp1()

Epoch 1/2
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3324 - sparse_categorical_accuracy: 0.8971
Epoch 2/2
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0839 - sparse_categorical_accuracy: 0.9735


In [7]:
#model = simple_mlp.train_mlp1()
simple_mlp.train_mlp()

Start of epoch 1
Step: 0, 손실: 116.073, 정확도: 6.250
Step: 100, 손실: 6.980, 정확도: 72.834
Step: 200, 손실: 4.156, 정확도: 77.254
Step: 300, 손실: 3.060, 정확도: 79.070
Step: 400, 손실: 2.426, 정확도: 80.977
Step: 500, 손실: 2.059, 정확도: 82.011
Step: 600, 손실: 1.788, 정확도: 83.221
Step: 700, 손실: 1.587, 정확도: 84.215
Step: 800, 손실: 1.442, 정확도: 84.929
Step: 900, 손실: 1.321, 정확도: 85.575
Step: 1000, 손실: 1.226, 정확도: 86.051
Step: 1100, 손실: 1.143, 정확도: 86.589
Step: 1200, 손실: 1.071, 정확도: 87.084
Step: 1300, 손실: 1.013, 정확도: 87.442
Step: 1400, 손실: 0.960, 정확도: 87.817
Step: 1500, 손실: 0.917, 정확도: 88.091
Step: 1600, 손실: 0.879, 정확도: 88.335
Step: 1700, 손실: 0.843, 정확도: 88.613
Step: 1800, 손실: 0.808, 정확도: 88.905
            epoch: 1,            Loss: 0.784,            Accuracy: 0.892,            
Start of epoch 2
Step: 0, 손실: 0.784, 정확도: 89.159
Step: 100, 손실: 0.756, 정확도: 89.426
Step: 200, 손실: 0.729, 정확도: 89.684
Step: 300, 손실: 0.706, 정확도: 89.868
Step: 400, 손실: 0.684, 정확도: 90.092
Step: 500, 손실: 0.665, 정확도: 90.267
Step: 600, 손실: 0.646, 정확